# prologue

### set up notebook and load package

In [1]:
# load what we need
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp

# data management

### datasets

Several datasets are available as pre-prepared containers that hold the data and some meta-data that is used in the algorithm
Any dataset can be turned into a container by invoking the constructor found in the file structures.py

In [2]:
# datasets might be down-sampled to make them easier to work with
# the full sets are available too
# this is a list of constructors that will be used in the benchmarking

# demo datasets that ship with package. all from UCI unless stated otherwise
# ds.adult_data, ds.adult_samp_data, ds.adult_small_samp_data Large dataset ships with manageable sub samples
# ds.bankmark_data, ds.bankmark_samp_data
# ds.car_data
# ds.cardio_data this is the cardiotocography dataset
# ds.credit_data
# ds.german_data
# ds.lending_data, ds.lending_samp_data, ds.lending_small_samp_data, ds.lending_tiny_samp_data from Kaggle. see datasets_from_source file for links
# ds.nursery_data, ds.nursery_samp_data
# ds.rcdv_data, ds.rcdv_samp_data from US government see datasets_from_source file for links
datasets = [
            ds.adult_small_samp,
            ds.bankmark_samp,
            ds.car,
            ds.cardio,
            ds.credit,
            ds.german,
            ds.lending_tiny_samp,
            ds.nursery_samp,
            ds.rcdv_samp
           ]

In [3]:
# example of one dataset
# note: random_state propagates through other functions and is easily updated to allow alternative runs
if False:
    ds.cardio(random_state=123, project_dir=project_dir)

### standardising train-test splitting
Some methods are not available in Python. We want to maintain the same dataset splits no matter which platform. So, the train test data is split with the one-time random seed and the splits are saved to csv in the project folders.

In [4]:
# writes external files
if False:
    rp.export_data_splits(datasets=datasets, project_dir=project_dir, random_state_splits=random_state_splits)

# Experimental Runs
Loop through datasets, actioning the functions in the package to execute a round of experiments and test evaluations.

## 0. Optional Memory and Computation Cost Management
CHIRPS is time economical but memory intensive to compute for lots of instances at once.


### Parallel processing
Scikit takes care of parallel for the RF construction.
We can parallelise the following:
1. the walk of instances down each tree to collect the paths. The paths for many instances are returned in a single array. This parallelises across trees.
2. building CHIRPS and the final explanation (rule). This is a search optimisation and we can parallelise each instance.

This is expecially effective when running batches. For single instances, set both to false to avoid spinning up the parallel infrastructure.

### Preparing unseen data

Again note:
test set has never been "seen" by random forest during training
test set has been only used to assess model (random forest) accuracy - no additional tuning after this
test set has not be involved in generating the explainer

#### Batching
The memory space requirements for all the paths can be reduced by dividing the test set into batches. However this does take longer as there is an overhead to instantiate all the required objects, especially if coupled with parallel processing.
Best compromise could be a small number of larger batches.

### 1. Data and Forest prep
Use the random state splits to do a one-off data split.
Fit the RF to training data, using the iterating random state.
Save the performance metrics on the test set for later review.

### 2. Prepare Unseen Data and Predictions
Important to note:
Test set never "seen" by RF during training.
test set not involved in generating the explainer.
Test set used to evaluate model (random forest) accuracy beyond OOBE scores - no additional tuning based on these results.
Test set used to evaluate explanation scores by leave-one-out method removing the specific instance we're explaining.

Important to note:
We will explain predictions directly from the trained RF. Explanation system makes no compromise on model accuracy.

### 3. CHIRPS algorithm
1. Extract tree prediction paths
2. Freqent pattern mining of paths
3. Score and sort mined path segments
4. Merge path segments into one rule

#### CHIRPS 1
Fit a forest_walker object to the dataset and decision forest. This is a wrapper that will extract the paths of all the given instances. Its main method delivers the instance paths for the remaining steps of the algorithm as a new object: a batch_paths_container. It can also report interesting statistics (treating the forest as a set of random tree-structured variables).

#### CHIRPS 2-4
A batch_CHIRPS_container is fitted with the batch_paths_container returned by the forest walker, and with a sample of data. For CHIRPS, we prefer a large sample. The whole training set or other representative sample will do. This is a wrapper object will execute steps 2-4 on all each the instance-paths in the batch_paths_container.

Important to note:
true_divide warnings are OK! It just means that a continuous variable is unbounded on one side i.e. no greater/less than inequality is used in the specific CHIRPS explanation.

Important note: 
Here we are using the training set to create the explainers. We could use a different dataset as long as it is representative of the training set that built the decision forest. Most important that we don't use the dataset that we wish to explain, so never use the test set, for example.

### 4. Evaluating CHIRPS Explanations
Test set has been used to create an explainer *one instance at a time* and the rest of test set was not "seen" during this construction. To score each explainer, we use test set, leaving out the individual instance being explained. The data_split_container (tt) has a convenience funtion for doing this. All the results are saved to csv files in the project directory.

In [5]:
# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.1

forest_walk_async=True
chirps_explanation_async=True

n_instances = 1

model = 'RandomForest'
# model = 'AdaBoost1'
# model = 'AdaBoost2'
# model = 'GBM'

do_Anchors = True
do_dfrgTrs = True

datasets = [datasets[0]] # here can opt for just one, e.g. datasets = [datasets[0]] (as an iterator)
start_instance = 0 # here can opt to start at a specific instance

# project_dir = '/datadisk/whiteboxing/examples'
project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

tuning = {'grid' : None, 'override' : False}
if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           do_raw=True, do_discretise=do_Anchors,
                                           start_instance=start_instance, verbose=verbose)

    kwargs = {'support_paths' : 0.1, 'alpha_paths' : 0.5, 'disc_path_bins' : 4,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control.update({'method' : 'Anchors'})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    if do_dfrgTrs:
        control.update({'method' : 'defragTrees',
                    'Kmax' : 1, 'restart' : 1, 'maxitr' : 1})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    
elif model == "AdaBoost1":
    algo = 'SAMME'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=True, do_discretise=do_Anchors,
                                       start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
             'support_paths' : 0.1, 'alpha_paths' : 0.0,
             'disc_path_bins' : 4, 'disc_path_eqcounts' : True,
             'score_func' : 1, 'weighting' : 'chisq',
             'merging_bootstraps' : merging_bootstraps,
             'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
 
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control.update({'method' : 'Anchors'})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    if do_dfrgTrs:
        control.update({'method' : 'defragTrees',
                    'Kmax' : 1, 'restart' : 1, 'maxitr' : 1})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    
elif model == 'AdaBoost2':
    algo = 'SAMME.R'
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                       random_state=random_state_rf,
                                       random_state_splits=random_state_splits,
                                       do_raw=True, do_discretise=do_Anchors,
                                       start_instance=start_instance, verbose=verbose)
    
    kwargs = {'paths_lengths_threshold' : 5,
                 'support_paths' : 0.01, 'alpha_paths' : 0.0,
                 'disc_path_bins' : 8, 'disc_path_eqcounts' : True,
                 'score_func' : 1, 'weighting' : 'kldiv',
                 'merging_bootstraps' : merging_bootstraps,
                 'pruning_bootstraps' : pruning_bootstraps, 'delta' : delta}
    
    control = {'method' : 'CHIRPS', 'model' : model,
                'n_instances' : n_instances,
                'random_state' : random_state_exp,
                'kwargs' : kwargs,
                'forest_walk_async' : forest_walk_async,
                'chirps_explanation_async' : chirps_explanation_async}
    
    rp.do_benchmarking(benchmark_items, verbose, **control)
    
    if do_Anchors:
        control.update({'method' : 'Anchors'})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    if do_dfrgTrs:
        control.update({'method' : 'defragTrees',
                    'Kmax' : 1, 'restart' : 1, 'maxitr' : 1})
        rp.do_benchmarking(benchmark_items, verbose, **control)
    
else: # GBM - not fully implemented yet
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)

Preprocessing adult_small_samp data and model for adult_small_samp with random state = 123
Split data into main train-test and build forest
Train main model
using previous tuning parameters


C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.8467
Best parameters:{'n_estimators': 400, 'oob_score': True, 'random_state': 123}

Discretise data and train model, e.g. for Anchors
using previous tuning parameters
Best OOB Accuracy Estimate during tuning: 0.8467
Best parameters:{'n_estimators': 400, 'oob_score': True, 'random_state': 123}


Beginning benchmark for adult_small_samp data.
Hyper-parameter settings: {'support_paths': 0.1, 'alpha_paths': 0.5, 'disc_path_bins': 4, 'score_func': 1, 'weighting': 'chisq', 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 1 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 0.7020 seconds

Running CHIRPS on a batch of 1 instances... (please wait)
CHIRPS time elapsed: 3.8113 seconds
CHIRPS with async = True

Evaluating found explanations
Results saved to V:\whiteboxing\tests\adult_small_samp\RandomForest_CHIRPS_rnst_123
CHIRPS

KeyboardInterrupt: 

In [ ]:
stop

## Sensitivity Analysis

In [ ]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp

# prepare data
datasets = [
            ds.adult_small_samp,
            ds.bankmark_samp,
            ds.car,
            ds.cardio,
            ds.credit,
            ds.german,
            ds.lending_tiny_samp,
            ds.nursery_samp,
            ds.rcdv_samp
           ]
# datasets = [datasets[0]] # for testing can just choose one


# CHIRPS default set up
merging_bootstraps = 20
pruning_bootstraps = 20
delta = 0.1 # prune rule terms if loss of precision no greater than delta

forest_walk_async=True
chirps_explanation_async=True

n_instances = 10000

# model = 'RandomForest'
# model = 'AdaBoost1'
model = 'AdaBoost2'
# model = 'GBM'

do_Anchors = False
do_dfrgTrs = False

# here can opt for just one, e.g. [datasets[0]] (as an iterator)
start_instance = 0 # here can opt to start at a specific instance

project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'
# project_dir = '/datadisk/whiteboxing'
random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

tuning = {'grid' : None, 'override' : False}

# for troubleshooting, can isolate an instance and do like
# bi_copy['cardio']['main']['ds_container'].current_row_test = 110

if model == 'RandomForest':
    tuning.update({'grid' : None}) # defaults to n_trees [200, 400, ..., 1600]
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance, verbose=verbose)
    
    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.1, 0.05], 18), 2)
    weighting = np.repeat(['chisq', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'disc_path_eqcounts' : False,
                        'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : 'majority',
                        'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}
    
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : 'rf_sensitivity'}


        rp.do_benchmarking(bi_copy, verbose, **control)
        
elif model in ('AdaBoost1', 'AdaBoost2'):
    if model == 'AdaBoost1':
        algo = 'SAMME'
        save_sensitivity_path = 'ada1_sensitivity'
        support_paths = np.tile(np.tile([0.05, 0.02, 0.01], 16), 2)
        n_kwargs = 36
    else:
        algo = 'SAMME.R'
        save_sensitivity_path = 'ada2_sensitivity'
        support_paths = np.tile(np.tile([0.005, 0.002, 0.001], 16), 2)
        n_kwargs = 72
        
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance, verbose=verbose)
    
    disc_path_bins = np.tile(np.tile(np.repeat([4, 8], 3), 6), 2)
    disc_path_eqcounts = np.tile(np.tile(np.repeat([True, False], 6), 6), 2)
    weighting = np.tile(np.repeat(['chisq', 'kldiv', 'nothing'], 12), 2)
    which_trees = np.repeat(['majority', 'conf_weighted'], 36)

    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                        'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                        'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : wchtr,
                        'delta' : delta} 
    for k, dpb, dpeq, w, sp, wchtr \
                   in zip(range(n_kwargs), disc_path_bins, disc_path_eqcounts, weighting, support_paths, which_trees)}

    for kwargs in kwargs_grid: # range(24, 36): #
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : save_sensitivity_path}
 
        rp.do_benchmarking(bi_copy, verbose, **control)

Preprocessing adult_small_samp data and model for adult_small_samp with random state = 123
Split data into main train-test and build forest
Train main model
using previous tuning parameters


C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.8473
Best parameters:{'algorithm': 'SAMME.R', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 200, 'random_state': 123}


Preprocessing bankmark_samp data and model for bankmark_samp with random state = 123
Split data into main train-test and build forest
Train main model
using previous tuning parameters


C:\ProgramData\Anaconda3\envs\B3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.9211
Best parameters:{'algorithm': 'SAMME.R', 'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), 'n_estimators': 400, 'random_state': 123}


Preprocessing car data and model for car with random state = 123
Split data into main train-test and build forest
Train main model
using previous tuning parameters


...

Beginning benchmark for credit data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 4, 'disc_path_eqcounts': False, 'score_func': 1, 'weighting': 'nothing', 'support_paths': 0.001, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'majority', 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 207 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 3.4723 seconds

Running CHIRPS on a batch of 207 instances... (please wait)
CHIRPS time elapsed: 101.2227 seconds
CHIRPS with async = True

Evaluating found explanations
Results saved to V:\whiteboxing\tests\credit\ada2_sensitivity\wcts_majority_sp_0.001_ap_0.0_dpb_4_dpeq_False_sf_1_w_nothing_AdaBoost2_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 4.6263 seconds

Beginning benchmark for german data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 4, 'di

Forest Walk with async = True
Forest Walk time elapsed: 1.4165 seconds

Running CHIRPS on a batch of 207 instances... (please wait)
CHIRPS time elapsed: 116.8958 seconds
CHIRPS with async = True

Evaluating found explanations
Results saved to V:\whiteboxing\tests\credit\ada2_sensitivity\wcts_majority_sp_0.005_ap_0.0_dpb_8_dpeq_False_sf_1_w_nothing_AdaBoost2_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 4.5497 seconds

Beginning benchmark for german data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 8, 'disc_path_eqcounts': False, 'score_func': 1, 'weighting': 'nothing', 'support_paths': 0.005, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'majority', 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 300 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 13.9202 seconds

Running CHIRPS on a batch of 300 instances... (please wait)
CHIRP

Forest Walk with async = True
Forest Walk time elapsed: 11.5431 seconds

Running CHIRPS on a batch of 300 instances... (please wait)
CHIRPS time elapsed: 540.4230 seconds
CHIRPS with async = True

Evaluating found explanations
Results saved to V:\whiteboxing\tests\german\ada2_sensitivity\wcts_majority_sp_0.002_ap_0.0_dpb_8_dpeq_False_sf_1_w_nothing_AdaBoost2_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 46.4834 seconds

Beginning benchmark for lending_tiny_samp data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 8, 'disc_path_eqcounts': False, 'score_func': 1, 'weighting': 'nothing', 'support_paths': 0.002, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'majority', 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 632 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 8.3332 seconds

Running CHIRPS on a batch of 632 instances... (please

Beginning benchmark for lending_tiny_samp data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 8, 'disc_path_eqcounts': False, 'score_func': 1, 'weighting': 'nothing', 'support_paths': 0.001, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'majority', 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 632 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 9.1635 seconds

Running CHIRPS on a batch of 632 instances... (please wait)
CHIRPS time elapsed: 429.4706 seconds
CHIRPS with async = True

Evaluating found explanations
Results saved to V:\whiteboxing\tests\lending_tiny_samp\ada2_sensitivity\wcts_majority_sp_0.001_ap_0.0_dpb_8_dpeq_False_sf_1_w_nothing_AdaBoost2_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 56.7078 seconds

Beginning benchmark for nursery_samp data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 

Forest Walk with async = True
Forest Walk time elapsed: 8.5521 seconds

Running CHIRPS on a batch of 632 instances... (please wait)
CHIRPS time elapsed: 1461.1543 seconds
CHIRPS with async = True

Evaluating found explanations
Results saved to V:\whiteboxing\tests\lending_tiny_samp\ada2_sensitivity\wcts_conf_weighted_sp_0.005_ap_0.0_dpb_4_dpeq_True_sf_1_w_chisq_AdaBoost2_CHIRPS_rnst_123
CHIRPS batch results eval time elapsed: 53.0956 seconds

Beginning benchmark for nursery_samp data.
Hyper-parameter settings: {'paths_lengths_threshold': 5, 'alpha_paths': 0.0, 'disc_path_bins': 4, 'disc_path_eqcounts': True, 'score_func': 1, 'weighting': 'chisq', 'support_paths': 0.005, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'conf_weighted', 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 778 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 15.4593 seconds

Running CHIRPS on a batch of 778 instances